# PREPARATIONS NEEDED TO RUN JOAKIM'S CODE
This script changes the dataframe to coincide with script requirements for running Wilminks scripts
- assumes that "BuildingDataset.ipynb" followed by "HS_MY_FilteringDataframe.ipynb" have been run to generate "MY_weather.csv", "MY_weather_filtered.csv" and "updateDF.csv"
- quick check of MeanTHI_adj at end of script to ensure have data

In [ ]:
from datetime import timedelta
import pandas as pd
import numpy as np

Make Year-Season

In [ ]:
# df = pd.read_csv('../Data/MergedData/MY_weather.csv', low_memory=False)
df = pd.read_csv('../Data/MergedData/MY_weather_filtered.csv', low_memory=False)
print(df.shape)
df['StartDate'] = pd.to_datetime(df['StartDate'])


# Function to determine the season based on month
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 1
    elif month in [3, 4, 5]:
        return 2
    elif month in [6, 7, 8]:
        return 3
    elif month in [9, 10, 11]:
        return 4


# Apply the function to create a 'YearSeason' variable
df['YearSeason'] = df['StartDate'].apply(lambda x: f"{x.year}0{get_season(x)}")

count_THI = df['THI_adj'].count()
print(count_THI)
count_THI = df['MeanTHI_adj'].count()  # Use MeanTHI_adj!
print(count_THI)

df["THI_adj2"] = df['THI_adj'].fillna(df['MeanTHI_adj'])
df["Temperature2"] = df['Temperature'].fillna(df['MeanTemperature'])

# reorder df - for filtered
col_keep = ["FarmName_Pseudo", 'SE_Number', 'AnimalNumber', 'Del_Cow_Id', 'Breed', 'CalvingDate', 'LactationNumber',
            'DaysInMilk2', 'StartDate', 'StartTime', 'SessionNumber', 'TotalYield', 'YearSeason',
            'Temperature2', 'THI_adj2',
            'RelativeHumidity', 'HW', 'cum_HW', 'Temp15Threshold']
df = df[col_keep]



# reorder df - for unfiltered
# col_keep = ["FarmName_Pseudo", 'SE_Number', 'AnimalNumber', 'Del_Cow_Id', 'Breed', 'CalvingDate', 'LactationNumber',
#            'DaysInMilk', 'StartDate', 'StartTime', 'SessionNumber', 'TotalYield', 'YearSeason',
#            'Temperature', 'RelativeHumidity', 'THI_adj', 'HW', 'cum_HW', 'Temp15Threshold']
# df = df[col_keep]

Add parentage

In [ ]:
df2 = pd.read_csv("../Data/MergedData/updateDF.csv", low_memory=False)
col_keep = ["SE_Number", "BirthDate", "Father_SE_Number", "Mother_SE_Number", "CullingDate"]
df2 = df2[col_keep]
df2 = df2.drop_duplicates(subset=["SE_Number"])

df3 = pd.merge(df, df2, on="SE_Number", how="left")

Make age

In [ ]:
df3['BirthDate'] = pd.to_datetime(df3['BirthDate'])
df3['StartDate'] = pd.to_datetime(df3['StartDate'])
df3['Age'] = (df3['StartDate'] - df3['BirthDate']).dt.days

# Filtered
df3.rename(columns={"CullingDate": "CullDecisionDate", "Father_SE_Number": "Father", "Mother_SE_Number": "Mother", "Breed": "BreedName", 
                    "DaysInMilk2": "DaysInMilk"},  inplace=True)

# Unfiltered
# df3.rename(columns={"CullingDate": "CullDecisionDate", "Father_SE_Number": "Father", "Mother_SE_Number": "Mother", "Breed": "BreedName"},  inplace=True)

Combine StartDate and StartTime

In [ ]:
# Make combined DateTime column
# Ensure StartDate is in datetime format
df3['StartDate'] = pd.to_datetime(df3['StartDate'], errors='coerce')

# Convert StartTime to a timedelta
df3["StartTime2"] = df3["StartTime"]
df3['StartTime2'] = pd.to_timedelta(df3['StartTime2'].astype(str))

# Combine StartDate and StartTime
df3['DateTime'] = df3['StartDate'] + df3['StartTime2']

Reorder dataframe

In [ ]:
new_order = ["FarmName_Pseudo", "SE_Number", "AnimalNumber", "StartDate", "StartTime", "LactationNumber", "DaysInMilk", "YearSeason",
             "TotalYield", "DateTime", "BreedName", "BirthDate", "Mother", "Father", "CullDecisionDate", "Temperature2", "RelativeHumidity",
             "THI_adj2", "HW", "cum_HW", "Temp15Threshold", "Age"]
df3 = df3[new_order]

# Filtered
df3.to_csv("../Data/MergedData/CleanedYieldData.csv", index=False)
df3.to_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241007/HeatStressEvaluationSummer24-1/Data/MergedData/CleanedYieldData.csv", index=False)

# Unfiltered
# df3.to_csv("../Data/MergedData/YieldData.csv", index=False)
# df3.to_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241007/HeatStressEvaluationSummer24-1/Data/MergedData/YieldData.csv", index=False)

Save

In [ ]:
# data = pd.read_csv('../Data/MergedData/YieldData.csv', low_memory=False)
df3 = pd.read_csv("../Data/MergedData/CleanedYieldData.csv", low_memory=False)
new_order = ['FarmName_Pseudo', 'SE_Number', 'AnimalNumber', 'StartDate', 'StartTime', 'DateTime', 'LactationNumber', 'DaysInMilk', 'YearSeason', 
             'TotalYield', 'DateTime', 'BreedName', 'Age', 'Mother', 'Father', 'CullDecisionDate', 'Temperature2', 'RelativeHumidity', 'THI_adj2',
             'HW', 'cum_HW', 'Temp15Threshold']
df3 = df3[new_order]
# df3.to_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241007/HeatStressEvaluationSummer24-1/Data/MergedData/YieldData.csv", index=False)
df3.to_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241007/HeatStressEvaluationSummer24-1/Data/MergedData/CleanedYieldData.csv", index=False)

In [ ]:
# Unfiltered
missing_count = df3["THI_adj"].isna().sum()
print(f"Number of missing values in 'THI_adj': {missing_count}")
print(df3.shape)

In [ ]:
# Filtered
missing_count = df3["THI_adj2"].isna().sum()
print(f"Number of missing values in 'THI_adj2': {missing_count}")
print(f"No. milking events in filtered file ready for Wilmink's script {df3.shape}")

Threshold for heat stress

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import statsmodels.api as sm
from sklearn.utils import resample

In [ ]:
# Load data
df = pd.read_csv("../Data/MergedData/CleanedYieldData.csv", low_memory=False)

In [ ]:
# Clean the data
df = df[['THI_adj2', 'TotalYield']].dropna()  # Select relevant columns and drop missing values
df = df.replace([np.inf, -np.inf], np.nan).dropna()  # Remove infinite values

In [ ]:
# Look at distributions
summary_stats = df['THI_adj2'].describe()
percentiles = np.percentile(df['THI_adj2'].dropna(), [1, 5, 10, 90, 95, 99])

print("Descriptive Statistics:\n", summary_stats)
print("\nPercentiles (1%, 5%, 10%, 90%, 95%, 99%):", percentiles)

In [ ]:
plt.hist(df['THI_adj2'], bins=50, color='blue', edgecolor='black')
plt.xlabel('THI')
plt.ylabel('Frequency')
plt.show()

Find Total Yield when THI is close to 0

If THI = 0 is less than the breakpoint, the first part of the piecewise_linear function applies:
TotalYield=intercept+slope1×THI

At THI=0THI=0:
TotalYield=intercept


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df['THI_adj2'], df['TotalYield'], alpha=0.7, edgecolor='k')

# Customize the plot
plt.title('THI vs. Total Milk Yield', fontsize=14)
plt.xlabel('THI (Temperature-Humidity Index)', fontsize=12)
plt.ylabel('Total Milk Yield (kg)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Define the piecewise linear function
def piecewise_linear(thi, intercept, slope1, slope2, breakpoint):
    return np.where(thi < breakpoint,
                    intercept + slope1 * thi,
                    intercept + slope1 * breakpoint + slope2 * (thi - breakpoint))

# Fit the model
popt, _ = curve_fit(piecewise_linear, df['THI_adj2'], df['TotalYield'], p0=[0.5, 0, -0.01, 61])

# Extract the optimal parameters from the fitted model
intercept, slope1, slope2, breakpoint = popt

# Calculate Total Yield when THI = 0
thi_value = 0
total_yield_at_thi_0 = piecewise_linear(thi_value, intercept, slope1, slope2, breakpoint)

print(f"Total Yield when THI = 0: {total_yield_at_thi_0}")

Estimate slopes

In [ ]:
# Define the breakpoint
breakpoint = 67

# Calculate slope1 (THI < breakpoint)
pre_break = df[df['THI_adj2'] < breakpoint]
slope1 = (pre_break['TotalYield'].max() - pre_break['TotalYield'].min()) / \
         (pre_break['THI_adj2'].max() - pre_break['THI_adj2'].min())

# Calculate slope2 (THI >= breakpoint)
post_break = df[df['THI_adj2'] >= breakpoint]
slope2 = (post_break['TotalYield'].max() - post_break['TotalYield'].min()) / \
         (post_break['THI_adj2'].max() - post_break['THI_adj2'].min())

print(f"Estimated slope1: {slope1}")
print(f"Estimated slope2: {slope2}")

Confirm slope estimation

In [ ]:
# Define the piecewise linear function
def piecewise_linear(thi, intercept, slope1, slope2, breakpoint):
    return np.where(thi < breakpoint,
                    intercept + slope1 * thi,
                    intercept + slope1 * breakpoint + slope2 * (thi - breakpoint))

# Define initial parameters
p0 = [12, 0.84, 2.73, 61]  # Example initial parameters: intercept, slope1, slope2, breakpoint

# Fit the model
popt, _ = curve_fit(piecewise_linear, df['THI_adj2'], df['TotalYield'], p0=p0)

# Output optimized parameters
print("Optimized Parameters:")
print(f"Intercept: {popt[0]:.2f}")
print(f"Slope1 (before breakpoint): {popt[1]:.2f}")
print(f"Slope2 (after breakpoint): {popt[2]:.2f}")
print(f"Breakpoint (THI threshold): {popt[3]:.2f}")

# Generate THI values for the plot
thi_values = np.linspace(df['THI_adj2'].min(), df['THI_adj2'].max(), 100)

# Compute predictions using initial and optimized parameters
initial_curve = piecewise_linear(thi_values, *p0)
optimized_curve = piecewise_linear(thi_values, *popt)

# Plot observed data
plt.scatter(df['THI_adj2'], df['TotalYield'], alpha=0.6, label='Observed Data', color='blue')

# Plot initial parameter curve
plt.plot(thi_values, initial_curve, color='orange', linestyle='--', label='Initial Parameters Curve')

# Plot optimized parameter curve
plt.plot(thi_values, optimized_curve, color='red', label='Optimized Parameters Curve')

# Add labels and legend
plt.title('THI vs. Total Milk Yield')
plt.xlabel('THI (Temperature-Humidity Index)')
plt.ylabel('Total Yield')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

Rerun with optimized slopes

In [ ]:
# Define the piecewise linear function
def piecewise_linear(thi, intercept, slope1, slope2, breakpoint):
    return np.where(thi < breakpoint,
                    intercept + slope1 * thi,
                    intercept + slope1 * breakpoint + slope2 * (thi - breakpoint))

# Define initial parameters
p0 = [12, -0.00, 0.07, 61]  # Example initial parameters: intercept, slope1, slope2, breakpoint

# Fit the model
popt, _ = curve_fit(piecewise_linear, df['THI_adj2'], df['TotalYield'], p0=p0)

# Generate THI values for the plot
thi_values = np.linspace(df['THI_adj2'].min(), df['THI_adj2'].max(), 100)

# Compute predictions using initial and optimized parameters
initial_curve = piecewise_linear(thi_values, *p0)
optimized_curve = piecewise_linear(thi_values, *popt)

# Plot observed data
plt.scatter(df['THI_adj2'], df['TotalYield'], alpha=0.6, label='Observed Data', color='blue')

# Plot initial parameter curve
plt.plot(thi_values, initial_curve, color='orange', linestyle='--', label='Initial Parameters Curve')

# Plot optimized parameter curve
plt.plot(thi_values, optimized_curve, color='red', label='Optimized Parameters Curve')

# Add labels and legend
plt.title('THI vs. Total Milk Yield')
plt.xlabel('THI (Temperature-Humidity Index)')
plt.ylabel('Total Yield')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

Find THI threshold to use

In [ ]:
# Define a piecewise linear function
def piecewise_linear(thi, intercept, slope1, slope2, breakpoint):
    return np.where(thi < breakpoint,
                    intercept + slope1 * thi,
                    intercept + slope1 * breakpoint + slope2 * (thi - breakpoint))

# Fit the model
popt, _ = curve_fit(piecewise_linear, df['THI_adj2'], df['TotalYield'], p0=[12.72, -0.00, 0.07, 67.18])

# Extract the estimated breakpoint (THI threshold)
estimated_threshold = popt[3]
print(f"Estimated THI Threshold: {estimated_threshold}")

# Plot the fitted curve
thi_values = np.linspace(df['THI_adj2'].min(), df['THI_adj2'].max(), 100)
predicted_cr = piecewise_linear(thi_values, *popt)

plt.scatter(df['THI_adj2'], df['TotalYield'], alpha=0.6, label='Observed Data')
plt.plot(thi_values, predicted_cr, color='red', label='Fitted Curve')
plt.axvline(x=estimated_threshold, color='green', linestyle='--', label='Estimated Threshold')
plt.xlabel('THI')
plt.ylabel('Total Yield')
plt.title('Piecewise Regression: THI vs. MY')
plt.legend()
plt.show()